In [0]:
from pyspark.sql import SparkSession
sc = SparkSession.builder.appName("Spark").getOrCreate()

In [0]:
data = sc.read.csv('/FileStore/tables/wc2018_players.csv', header=True, inferSchema=True)
data.show(1)

+---------+---+----+------------------+----------+----------+--------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|          Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|AFC Ajax (NED)|   169|    65|
+---------+---+----+------------------+----------+----------+--------------+------+------+
only showing top 1 row



In [0]:
# Create a view with only name and orginal height+1
data.select('FIFA Popular Name', data.Height + 1).show(1)

+------------------+------------+
| FIFA Popular Name|(Height + 1)|
+------------------+------------+
|TAGLIAFICO Nicolas|         170|
+------------------+------------+
only showing top 1 row



In [0]:
# Show only player's name whose height > 170
data.select('Shirt Name', data['Height']>170).show(5)

+----------+--------------+
|Shirt Name|(Height > 170)|
+----------+--------------+
|TAGLIAFICO|         false|
|     PAVÓN|         false|
|   LANZINI|         false|
|    SALVIO|         false|
|     MESSI|         false|
+----------+--------------+
only showing top 5 rows



In [0]:
# Show FIFA Popular Name and 0/1 depending on Height > 170
data.select( 'FIFA Popular Name' ,(data['Height'] > 170).cast('integer').alias('Height') ).show(10)

+------------------+------+
| FIFA Popular Name|Height|
+------------------+------+
|TAGLIAFICO Nicolas|     0|
|    PAVON Cristian|     0|
|    LANZINI Manuel|     0|
|    SALVIO Eduardo|     0|
|      MESSI Lionel|     0|
|  ANSALDI Cristian|     1|
|      BIGLIA Lucas|     1|
|       BANEGA Ever|     1|
| MASCHERANO Javier|     1|
|      DYBALA Paulo|     1|
+------------------+------+
only showing top 10 rows



In [0]:
# Name of the Shortest Player
from pyspark.sql.functions import min
minHeight = data.select( min(data['Height']) ).show()
data.select('FIFA Popular Name').filter( data['Height'] == 165 ).show()
data.sort('Height').show(3)

+-----------+
|min(Height)|
+-----------+
|        165|
+-----------+

+-----------------+
|FIFA Popular Name|
+-----------------+
| QUINTERO Alberto|
|   YAHIA ALSHEHRI|
|  SHAQIRI Xherdan|
+-----------------+

+------------+---+----+-----------------+----------+----------+--------------------+------+------+
|        Team|  #|Pos.|FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+------------+---+----+-----------------+----------+----------+--------------------+------+------+
|Saudi Arabia|  8|  MF|   YAHIA ALSHEHRI|26.06.1990|     YAHIA|    CD Leganés (ESP)|   165|    63|
| Switzerland| 23|  MF|  SHAQIRI Xherdan|10.10.1991|   SHAQIRI| Stoke City FC (ENG)|   165|    78|
|      Panama| 19|  MF| QUINTERO Alberto|18.12.1987|  QUINTERO|Universitario Lim...|   165|    62|
+------------+---+----+-----------------+----------+----------+--------------------+------+------+
only showing top 3 rows



In [0]:
# Find the tallest player
from pyspark.sql.functions import max, col
maxHeight = data.select( max(data['Height']) )
data.select('FIFA Popular Name').filter( data['Height'] == 201 ).show()
data.sort('Height', ascending = False).show(1)

+-----------------+
|FIFA Popular Name|
+-----------------+
|    KALINIC Lovre|
+-----------------+

+-------+---+----+-----------------+----------+----------+--------------+------+------+
|   Team|  #|Pos.|FIFA Popular Name|Birth Date|Shirt Name|          Club|Height|Weight|
+-------+---+----+-----------------+----------+----------+--------------+------+------+
|Croatia| 12|  GK|    KALINIC Lovre|03.04.1990|L. KALINIĆ|KAA Gent (BEL)|   201|    96|
+-------+---+----+-----------------+----------+----------+--------------+------+------+
only showing top 1 row



In [0]:
# Average Height of the players on the Argentina team
from pyspark.sql.functions import mean
data.select(mean(data['Height'])).show()

+-----------------+
|      avg(Height)|
+-----------------+
|182.4076086956522|
+-----------------+



In [0]:
# LEVEL 2 Questions
movies = sc.read.csv('/FileStore/tables/movies.csv', header = True, inferSchema = True)
ratings = sc.read.csv('/FileStore/tables/ratings.csv', header = True, inferSchema = True)

In [0]:
movies.createOrReplaceTempView('movies')
ratings.createOrReplaceTempView('ratings')
movies.show(5)
ratings.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
+------+-------+------+----------+
only showing top 5 rows



In [0]:
# Find the list of oldest released movies
from pyspark.sql.functions import col
movies = sc.sql("select movieId, title, genres, CAST(substring(right(title,5),1, 4) as Integer) as year from movies where title like '%(%)'")
movies.createOrReplaceTempView('movies')

sc.sql("select * from movies order by year").show(10)
print( movies.count() )
movies = movies.dropna()
print( movies.count() )

movies.createOrReplaceTempView('movies')
sc.sql("select * from movies order by year").show(10)

+-------+--------------------+--------------------+----+
|movieId|               title|              genres|year|
+-------+--------------------+--------------------+----+
| 108548|Big Bang Theory, ...|              Comedy|null|
|  32898|Trip to the Moon,...|Action|Adventure|...|1902|
|   7065|Birth of a Nation...|           Drama|War|1915|
|   7243|Intolerance: Love...|               Drama|1916|
|  62383|20,000 Leagues Un...|Action|Adventure|...|1916|
|   8511|Immigrant, The (1...|              Comedy|1917|
|   3309|Dog's Life, A (1918)|              Comedy|1918|
|  72626|Billy Blazes, Esq...|      Comedy|Western|1919|
|  59945|Mark of Zorro, Th...|Adventure|Romance...|1920|
|  25737|Golem, The (Golem...|      Fantasy|Horror|1920|
+-------+--------------------+--------------------+----+
only showing top 10 rows

9114
9113
+-------+--------------------+--------------------+----+
|movieId|               title|              genres|year|
+-------+--------------------+--------------------+-

In [0]:
# How many movies are released each year?
sc.sql("select year, count(*) from movies group by year order by year").show(5)

+----+--------+
|year|count(1)|
+----+--------+
|1902|       1|
|1915|       1|
|1916|       2|
|1917|       1|
|1918|       1|
+----+--------+
only showing top 5 rows



In [0]:
# How many number of movies are there for each rating?
sc.sql("select rating, count(ratings.movieId) as num_movies from ratings group by rating order by rating").show(5)

+------+----------+
|rating|num_movies|
+------+----------+
|   0.5|      1101|
|   1.0|      3326|
|   1.5|      1687|
|   2.0|      7271|
|   2.5|      4449|
+------+----------+
only showing top 5 rows



In [0]:
# How many users have rated each movie?
print(movies.count())
print(ratings.count())
data = movies.join(ratings, on="movieId", how='outer')
print( data.count() )
data.createOrReplaceTempView('movieRatings')
sc.sql("select title, count(userId) from movieRatings group by title").show(5)

9113
100004
100062
+--------------------+-------------+
|               title|count(userId)|
+--------------------+-------------+
|    Fair Game (1995)|            7|
| If Lucy Fell (1996)|            5|
| Three Wishes (1995)|            1|
|Heavenly Creature...|           43|
|Paris, France (1993)|            1|
+--------------------+-------------+
only showing top 5 rows



In [0]:
# What is the total rating for each movie?
sc.sql("select title, sum(rating) from movieRatings group by title").show(5)

+--------------------+-----------+
|               title|sum(rating)|
+--------------------+-----------+
|    Fair Game (1995)|       17.0|
| If Lucy Fell (1996)|       14.0|
| Three Wishes (1995)|        4.0|
|Heavenly Creature...|      157.0|
|Paris, France (1993)|        5.0|
+--------------------+-----------+
only showing top 5 rows



In [0]:
# What is the average rating for each movie?
sc.sql("select title, avg(rating) from movieRatings group by title").show(5)

+--------------------+------------------+
|               title|       avg(rating)|
+--------------------+------------------+
|    Fair Game (1995)|2.4285714285714284|
| If Lucy Fell (1996)|               2.8|
| Three Wishes (1995)|               4.0|
|Heavenly Creature...|3.6511627906976742|
|Paris, France (1993)|               5.0|
+--------------------+------------------+
only showing top 5 rows

